In [ ]:
# -*- coding: utf-8 -*-
"""Pertemuan13_Evaluasi_Model_Regresi_FIXED.ipynb

Automatically generated by Colab.
**Pertemuan 13: Evaluasi Model Regresi - FIXED VERSION**
MAE, MSE, RMSE, R², MAPE, Residual Analysis, Cross-Validation
"""

# Install and import stable libraries (Yellowbrick for viz)
!pip install yellowbrick -q
!pip install plotly -q

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning and Metrics
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                           r2_score, mean_absolute_percentage_error)
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# Visualization Libraries (Stable ones)
from yellowbrick.regressor import ResidualsPlot, PredictionError
import plotly.graph_objects as go
from plotly.subplots import make_subplots

print("Libraries loaded successfully!")

In [ ]:
# =============================================
# BAGIAN 1: DATASET & PREPROCESSING
# =============================================

print("BAGIAN 1: DATASET & PREPROCESSING")
print("=" * 60)

In [ ]:
# Langkah 1: Load dataset regresi - California Housing Prices
print("DATASET: CALIFORNIA HOUSING PRICES")
print("Dataset ini cocok untuk evaluasi model regresi karena:")
print("• Variabel target kontinu (harga rumah)")
print("• Multiple features")
print("• Real-world dataset")

from sklearn.datasets import fetch_california_housing

# Load dataset
housing = fetch_california_housing()
X = pd.DataFrame(housing.data, columns=housing.feature_names)
y = pd.Series(housing.target, name='MedHouseVal')  # Median house value (dalam $100,000s)

print(f"\n INFORMASI DATASET:")
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")
print(f"Target: {y.name}")

# Tampilkan 5 data pertama
print("\n 5 DATA PERTAMA:")
print(pd.concat([X.head(), y.head()], axis=1))

In [ ]:
# Langkah 2: Exploratory Data Analysis

print("\n EXPLORATORY DATA ANALYSIS")

# Statistik deskriptif
print("STATISTIK DESKRIPTIF FEATURES:")
print(X.describe())

print("\n STATISTIK DESKRIPTIF TARGET:")
print(f"Mean: {y.mean():.4f}")
print(f"Std: {y.std():.4f}")
print(f"Min: {y.min():.4f}")
print(f"25%: {y.quantile(0.25):.4f}")
print(f"50%: {y.median():.4f}")
print(f"75%: {y.quantile(0.75):.4f}")
print(f"Max: {y.max():.4f}")

# Visualisasi distribusi target
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
sns.histplot(y, bins=50, kde=True, color='skyblue')
plt.title('Distribusi Harga Rumah (Target)')
plt.xlabel('Median House Value ($100,000)')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
sns.boxplot(y=y, color='lightgreen')
plt.title('Boxplot Harga Rumah')
plt.xlabel('Median House Value ($100,000)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
from scipy import stats
stats.probplot(y, dist="norm", plot=plt)
plt.title('Q-Q Plot (Test Normalitas)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Korelasi features dengan target
print("\n🔗 KORELASI FEATURES DENGAN TARGET:")
correlations = X.copy()
correlations['target'] = y
corr_with_target = correlations.corr()['target'].sort_values(ascending=False)[1:]  # Exclude self-correlation

print(corr_with_target)

# Visualisasi korelasi
plt.figure(figsize=(10, 6))
sns.barplot(x=corr_with_target.values, y=corr_with_target.index, palette='coolwarm')
plt.title('Korelasi Features dengan Harga Rumah')
plt.xlabel('Correlation Coefficient')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# Langkah 3: Data Preprocessing untuk Modeling

print("\n DATA PREPROCESSING")

# Split data: 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"DATA SPLITTING:")
print(f"Train set: {X_train.shape} samples")
print(f"Test set: {X_test.shape} samples")
print(f"Train target mean: {y_train.mean():.4f}")
print(f"Test target mean: {y_test.mean():.4f}")

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n PREPROCESSING SELESAI!")
print(f"Features setelah scaling: Mean ≈ 0, Std ≈ 1")

In [ ]:
# =============================================
# BAGIAN 2: TRAINING MULTIPLE REGRESSION MODELS
# =============================================

print("\n BAGIAN 2: TRAINING MULTIPLE REGRESSION MODELS")
print("=" * 60)

In [ ]:
# Langkah 4: Inisialisasi dan Training Multiple Regression Models

print("INISIALISASI MODEL REGRESI")

# Define models dengan parameter default
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(alpha=1.0, random_state=42),
    'Lasso Regression': Lasso(alpha=0.1, random_state=42),
    'Decision Tree': DecisionTreeRegressor(random_state=42, max_depth=5),
    'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42, n_estimators=100),
    'Support Vector Regressor': SVR(kernel='rbf'),
    'K-Neighbors Regressor': KNeighborsRegressor(n_neighbors=5)
}

# Dictionary untuk menyimpan hasil
results = {
    'Model': [],
    'MAE': [],
    'MSE': [],
    'RMSE': [],
    'R2': [],
    'Adj_R2': [],
    'MAPE': [],
    'Explained_Variance': []
}

# Dictionary untuk menyimpan model yang sudah trained
trained_models = {}
predictions = {}
residuals = {}

print("TRAINING MODEL REGRESI...")
print("-" * 50)

for name, model in models.items():
    print(f"Training {name}...")

    try:
        # Train model
        model.fit(X_train_scaled, y_train)
        trained_models[name] = model

        # Predictions
        y_pred = model.predict(X_test_scaled)
        predictions[name] = y_pred

        # Calculate residuals
        residuals[name] = y_test - y_pred

        # Calculate all metrics
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        # Adjusted R²
        n = len(y_test)
        p = X_test.shape[1]
        adj_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

        # MAPE (handle zero division)
        try:
            mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # Convert to percentage
        except:
            mape = np.nan

        # Explained Variance
        explained_var = explained_variance_score(y_test, y_pred)

        # Store results
        results['Model'].append(name)
        results['MAE'].append(mae)
        results['MSE'].append(mse)
        results['RMSE'].append(rmse)
        results['R2'].append(r2)
        results['Adj_R2'].append(adj_r2)
        results['MAPE'].append(mape)
        results['Explained_Variance'].append(explained_var)

        print(f"  {name} - R²: {r2:.4f}, RMSE: {rmse:.4f}")

    except Exception as e:
        print(f"  Error training {name}: {str(e)}")
        # Skip this model

print("\n SEMUA MODEL SELESAI DITRAINING!")

In [ ]:
# Langkah 5: Tampilkan Perbandingan Hasil Model

print("\n PERBANDINGAN HASIL MODEL REGRESI")
print("=" * 60)

# Convert results ke DataFrame
results_df = pd.DataFrame(results)

# Sort berdasarkan R² (descending)
results_df = results_df.sort_values('R2', ascending=False)

# Format untuk tampilan yang lebih baik
display_df = results_df.copy()
numeric_cols = ['MAE', 'MSE', 'RMSE', 'R2', 'Adj_R2', 'MAPE', 'Explained_Variance']

for col in numeric_cols:
    if col == 'MAPE':
        # Format MAPE as percentage
        display_df[col] = display_df[col].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "N/A")
    else:
        display_df[col] = display_df[col].apply(lambda x: f"{x:.4f}")

print("PERINGKAT MODEL BERDASARKAN R²:")
print(display_df.to_string(index=False))

# Visualisasi perbandingan metrics
plt.figure(figsize=(15, 12))

metrics_to_plot = ['MAE', 'RMSE', 'R2', 'Adj_R2']
colors = plt.cm.Set2(np.linspace(0, 1, len(metrics_to_plot)))

for idx, metric in enumerate(metrics_to_plot):
    plt.subplot(2, 2, idx+1)

    # Sort berdasarkan metric (ascending untuk error metrics, descending untuk R²)
    if metric in ['MAE', 'RMSE']:
        sorted_idx = results_df[metric].argsort()
        title_suffix = '(Lower is Better)'
    else:
        sorted_idx = results_df[metric].argsort()[::-1]  # Descending
        title_suffix = '(Higher is Better)'

    plt.barh(range(len(models)), results_df[metric].iloc[sorted_idx],
             color=colors[idx])
    plt.yticks(range(len(models)), results_df['Model'].iloc[sorted_idx])
    plt.xlabel(metric)
    plt.title(f'{metric} Comparison {title_suffix}')
    plt.grid(True, alpha=0.3, axis='x')

    # Tambahkan nilai pada bar
    for i, v in enumerate(results_df[metric].iloc[sorted_idx]):
        plt.text(v + (0.01 if metric in ['R2', 'Adj_R2'] else -0.01), i,
                f'{v:.3f}', va='center',
                ha='left' if metric in ['R2', 'Adj_R2'] else 'right',
                fontsize=9)

plt.tight_layout()
plt.show()

# Heatmap perbandingan metrics
plt.figure(figsize=(12, 8))
heatmap_data = results_df.set_index('Model')[['MAE', 'RMSE', 'R2', 'Adj_R2', 'Explained_Variance']]

# Normalize untuk heatmap (kecuali R² dan Adj_R² yang sudah 0-1)
normalized_data = heatmap_data.copy()
for col in ['MAE', 'RMSE']:
    normalized_data[col] = 1 - (heatmap_data[col] - heatmap_data[col].min()) / (heatmap_data[col].max() - heatmap_data[col].min())

sns.heatmap(normalized_data, annot=heatmap_data.round(3), fmt='.3f',
            cmap='RdYlGn', linewidths=1, linecolor='black',
            cbar_kws={'label': 'Normalized Performance (Green=Better)'})
plt.title('Model Performance Comparison Heatmap\n(Original values in cells)')
plt.tight_layout()
plt.show()

In [ ]:
# =============================================
# BAGIAN 3: DETAILED MODEL EVALUATION
# =============================================

print("\n BAGIAN 3: DETAILED MODEL EVALUATION")
print("=" * 60)

In [ ]:
# Langkah 6: Analisis Actual vs Predicted

print("ACTUAL VS PREDICTED ANALYSIS")

# Pilih model terbaik berdasarkan R²
best_model_name = results_df.iloc[0]['Model']
best_model = trained_models[best_model_name]
best_predictions = predictions[best_model_name]

print(f"\n ANALISIS UNTUK MODEL TERBAIK: {best_model_name}")
print("-" * 50)

# Actual vs Predicted plot
plt.figure(figsize=(15, 5))

# Plot 1: Scatter plot actual vs predicted
plt.subplot(1, 3, 1)
plt.scatter(y_test, best_predictions, alpha=0.6, color='blue')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
         'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title(f'Actual vs Predicted\n{best_model_name}')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Prediction error distribution
plt.subplot(1, 3, 2)
errors = y_test - best_predictions
sns.histplot(errors, bins=50, kde=True, color='red')
plt.axvline(x=0, color='black', linestyle='--', linewidth=1)
plt.xlabel('Prediction Error (Actual - Predicted)')
plt.ylabel('Frequency')
plt.title('Error Distribution')
plt.grid(True, alpha=0.3)

# Plot 3: Prediction error vs actual
plt.subplot(1, 3, 3)
plt.scatter(y_test, errors, alpha=0.6, color='green')
plt.axhline(y=0, color='black', linestyle='--', linewidth=1)
plt.xlabel('Actual Values')
plt.ylabel('Prediction Error')
plt.title('Error vs Actual Values')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistik error
print("\n ERROR STATISTICS:")
print(f"Mean Error: {errors.mean():.4f}")
print(f"Std Error: {errors.std():.4f}")
print(f"Min Error: {errors.min():.4f}")
print(f"Max Error: {errors.max():.4f}")
print(f"Error Range: {errors.max() - errors.min():.4f}")
print(f"Percentage within ±0.1: {(abs(errors) <= 0.1).mean()*100:.1f}%")
print(f"Percentage within ±0.2: {(abs(errors) <= 0.2).mean()*100:.1f}%")
print(f"Percentage within ±0.5: {(abs(errors) <= 0.5).mean()*100:.1f}%")

# Business interpretation
print("\n BUSINESS INTERPRETATION (dalam $100,000):")
print(f"Rata-rata model meleset: ${abs(errors.mean())*100000:.0f}")
print(f"Standard deviation error: ${errors.std()*100000:.0f}")
print(f"68% prediksi meleset dalam ±${errors.std()*100000:.0f}")
print(f"95% prediksi meleset dalam ±${2*errors.std()*100000:.0f}")

# Compare untuk semua model
print("\n COMPARISON ACTUAL vs PREDICTED UNTUK SEMUA MODEL")

# Plot untuk semua model
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for idx, (name, y_pred) in enumerate(predictions.items()):
    if idx < len(axes):
        ax = axes[idx]
        ax.scatter(y_test, y_pred, alpha=0.5, s=10)
        ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()],
                'r--', lw=1)
        ax.set_xlabel('Actual')
        ax.set_ylabel('Predicted')
        ax.set_title(f'{name}\nR²: {r2_score(y_test, y_pred):.3f}')
        ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Langkah 7: Analisis Residual

print("\n RESIDUAL ANALYSIS")

# Residual analysis untuk model terbaik
print(f"\n RESIDUAL ANALYSIS UNTUK {best_model_name}")

# Residual plots
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Plot 1: Residuals vs Predicted
axes[0, 0].scatter(best_predictions, residuals[best_model_name], alpha=0.6)
axes[0, 0].axhline(y=0, color='r', linestyle='--')
axes[0, 0].set_xlabel('Predicted Values')
axes[0, 0].set_ylabel('Residuals')
axes[0, 0].set_title('Residuals vs Predicted')
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Residuals vs Actual
axes[0, 1].scatter(y_test, residuals[best_model_name], alpha=0.6)
axes[0, 1].axhline(y=0, color='r', linestyle='--')
axes[0, 1].set_xlabel('Actual Values')
axes[0, 1].set_ylabel('Residuals')
axes[0, 1].set_title('Residuals vs Actual')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Q-Q plot residuals
from scipy import stats
stats.probplot(residuals[best_model_name], dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('Q-Q Plot Residuals')
axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Histogram residuals
axes[1, 0].hist(residuals[best_model_name], bins=50, density=True, alpha=0.7, color='blue')
# Fit normal distribution
from scipy.stats import norm
mu, std = norm.fit(residuals[best_model_name])
xmin, xmax = axes[1, 0].get_xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
axes[1, 0].plot(x, p, 'k', linewidth=2, label=f'N({mu:.3f}, {std:.3f}²)')
axes[1, 0].set_xlabel('Residuals')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title(f'Residual Distribution\nShapiro p-value: {stats.shapiro(residuals[best_model_name])[1]:.4f}')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 5: Residuals over index (check for patterns)
axes[1, 1].plot(residuals[best_model_name].values, 'o', alpha=0.6, markersize=3)
axes[1, 1].axhline(y=0, color='r', linestyle='--')
axes[1, 1].set_xlabel('Observation Index')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Residuals over Index')
axes[1, 1].grid(True, alpha=0.3)

# Plot 6: Autocorrelation of residuals
from statsmodels.graphics.tsaplots import plot_acf
plot_acf(residuals[best_model_name], lags=20, ax=axes[1, 2], alpha=0.05)
axes[1, 2].set_title('Residuals Autocorrelation')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretasi residual analysis
print("\n INTERPRETASI RESIDUAL ANALYSIS:")

# 1. Check randomness (Residuals vs Predicted)
corr_res_pred = np.corrcoef(best_predictions, residuals[best_model_name])[0, 1]
print(f"1. Correlation residuals-predicted: {corr_res_pred:.4f}")
if abs(corr_res_pred) < 0.1:
    print("   Good: Residuals tidak berkorelasi dengan predicted values")
else:
    print(f"   Warning: Residuals berkorelasi dengan predicted values (|r| = {abs(corr_res_pred):.2f})")

# 2. Check constant variance
# Split residuals into groups based on predicted values
n_groups = 5
pred_groups = pd.qcut(best_predictions, n_groups, labels=False)
residual_variance = [residuals[best_model_name][pred_groups == i].var() for i in range(n_groups)]
variance_ratio = max(residual_variance) / min(residual_variance) if min(residual_variance) > 0 else np.inf
print(f"\n2. Variance ratio across groups: {variance_ratio:.2f}")
if variance_ratio < 4:
    print("   Good: Variance relatif konstan (homoscedasticity)")
else:
    print("   Warning: Variance tidak konstan (heteroscedasticity)")

# 3. Check normality
shapiro_test = stats.shapiro(residuals[best_model_name])
print(f"\n3. Shapiro-Wilk normality test: p-value = {shapiro_test[1]:.4f}")
if shapiro_test[1] > 0.05:
    print("   Good: Residuals terdistribusi normal")
else:
    print("   Warning: Residuals tidak terdistribusi normal")

# 4. Check independence (autocorrelation)
from statsmodels.stats.stattools import durbin_watson
dw_stat = durbin_watson(residuals[best_model_name])
print(f"\n4. Durbin-Watson statistic: {dw_stat:.3f}")
if 1.5 < dw_stat < 2.5:
    print("   Good: Residuals independent (no autocorrelation)")
elif dw_stat <= 1.5:
    print("   Warning: Positive autocorrelation")
else:
    print("   Warning: Negative autocorrelation")

print("\n RECOMMENDATIONS BASED ON RESIDUAL ANALYSIS:")
issues = []
if abs(corr_res_pred) > 0.1:
    issues.append("• Model mungkin non-linear, pertimbangkan polynomial features")
if variance_ratio >= 4:
    issues.append("• Transformasi target (log, sqrt) mungkin diperlukan")
if shapiro_test[1] <= 0.05:
    issues.append("• Robust regression methods mungkin lebih baik")
if dw_stat <= 1.5 or dw_stat >= 2.5:
    issues.append("• Time series effects mungkin ada, pertimbangkan lag features")

if issues:
    for issue in issues:
        print(issue)
else:
    print("Model memenuhi semua asumsi regresi linear klasik")

In [ ]:
# Langkah 8: Deep Dive Error Metrics

print("\n ERROR METRICS DEEP DIVE")

# Comparative analysis semua metrics
print("COMPARATIVE METRICS ANALYSIS UNTUK SEMUA MODEL")

# Buat DataFrame untuk analysis
metrics_df = results_df.copy()

# Tambahkan ranking untuk setiap metric
metrics_df['MAE_Rank'] = metrics_df['MAE'].rank(ascending=True)
metrics_df['RMSE_Rank'] = metrics_df['RMSE'].rank(ascending=True)
metrics_df['R2_Rank'] = metrics_df['R2'].rank(ascending=False)  # Higher is better
metrics_df['MAPE_Rank'] = metrics_df['MAPE'].rank(ascending=True)

# Hitung average rank
metrics_df['Avg_Rank'] = metrics_df[['MAE_Rank', 'RMSE_Rank', 'R2_Rank']].mean(axis=1)
metrics_df['Overall_Rank'] = metrics_df['Avg_Rank'].rank(ascending=True)

print("\n RANKING MODEL BERDASARKAN BERBAGAI METRICS:")
rank_cols = ['Model', 'MAE_Rank', 'RMSE_Rank', 'R2_Rank', 'Avg_Rank', 'Overall_Rank']
print(metrics_df[rank_cols].sort_values('Overall_Rank').to_string(index=False))

# Visualisasi metrics trade-off
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: MAE vs RMSE trade-off
scatter1 = axes[0, 0].scatter(metrics_df['MAE'], metrics_df['RMSE'],
                              s=metrics_df['R2']*500, c=metrics_df['R2'],
                              cmap='RdYlGn', alpha=0.7, edgecolors='black')
axes[0, 0].set_xlabel('MAE (Lower is Better)')
axes[0, 0].set_ylabel('RMSE (Lower is Better)')
axes[0, 0].set_title('MAE vs RMSE Trade-off\n(Size = R², Color = R²)')
plt.colorbar(scatter1, ax=axes[0, 0], label='R² Score')
# Annotate points
for idx, row in metrics_df.iterrows():
    axes[0, 0].annotate(row['Model'], (row['MAE'], row['RMSE']),
                       xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: R² vs RMSE trade-off
scatter2 = axes[0, 1].scatter(metrics_df['R2'], metrics_df['RMSE'],
                              s=metrics_df['Explained_Variance']*500,
                              c=metrics_df['MAE'], cmap='viridis_r',
                              alpha=0.7, edgecolors='black')
axes[0, 1].set_xlabel('R² (Higher is Better)')
axes[0, 1].set_ylabel('RMSE (Lower is Better)')
axes[0, 1].set_title('R² vs RMSE Trade-off\n(Size = Explained Variance, Color = MAE)')
plt.colorbar(scatter2, ax=axes[0, 1], label='MAE')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Model complexity vs performance
# Estimate complexity (number of parameters/features used)
complexity_estimate = []
for name in metrics_df['Model']:
    if 'Linear' in name or 'Ridge' in name or 'Lasso' in name:
        complexity_estimate.append(1)  # Linear models
    elif 'Tree' in name:
        complexity_estimate.append(3)  # Medium complexity
    elif 'Forest' in name or 'Boosting' in name:
        complexity_estimate.append(5)  # High complexity
    elif 'SVR' in name:
        complexity_estimate.append(4)  # Medium-high complexity
    elif 'KNN' in name:
        complexity_estimate.append(2)  # Low-medium complexity
    else:
        complexity_estimate.append(3)  # Default

metrics_df['Complexity_Estimate'] = complexity_estimate

# Create a numerical index for each model for coloring
unique_models = metrics_df['Model'].unique()
model_to_color_index = {name: i for i, name in enumerate(unique_models)}
metrics_df['Model_Color_Index'] = metrics_df['Model'].map(model_to_color_index)

scatter3 = axes[1, 0].scatter(metrics_df['Complexity_Estimate'], metrics_df['R2'],
                              s=metrics_df['RMSE']*100, c=metrics_df['Model_Color_Index'],
                              cmap='tab20', # Use the colormap name
                              alpha=0.7, edgecolors='black')
axes[1, 0].set_xlabel('Model Complexity Estimate')
axes[1, 0].set_ylabel('R² Score')
axes[1, 0].set_title('Complexity vs Performance\n(Size = RMSE)')

# Create legend for model names and colors
handles = []
labels = []
colors = plt.cm.get_cmap('tab20', len(unique_models)) # Get the colormap
for i, model_name in enumerate(unique_models):
    handles.append(plt.Line2D([0], [0], marker='o', color='w',
                              markerfacecolor=colors(i), markersize=10, label=model_name))
    labels.append(model_name)
axes[1, 0].legend(handles=handles, labels=labels, title="Models", bbox_to_anchor=(1.05, 1), loc='upper left')

# Annotate points
for idx, row in metrics_df.iterrows():
    axes[1, 0].annotate(row['Model'], (row['Complexity_Estimate'], row['R2']),
                       xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Metrics correlation heatmap
metrics_corr = metrics_df[['MAE', 'RMSE', 'R2', 'Adj_R2', 'MAPE', 'Explained_Variance']].corr()
mask = np.triu(np.ones_like(metrics_corr, dtype=bool))
sns.heatmap(metrics_corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, ax=axes[1, 1], square=True)
axes[1, 1].set_title('Correlation between Error Metrics')

plt.tight_layout()
plt.show()

# Business interpretation of error metrics
print("\n BUSINESS INTERPRETATION ERROR METRICS:")

# Konversi ke dollar
conversion_factor = 100000  # $100,000 per unit

for idx, row in metrics_df.iterrows():
    model_name = row['Model']
    mae_dollars = row['MAE'] * conversion_factor
    rmse_dollars = row['RMSE'] * conversion_factor

    print(f"\n{model_name}:")
    print(f"  • MAE: ${mae_dollars:,.0f} (rata-rata error absolut)")
    print(f"  • RMSE: ${rmse_dollars:,.0f} (std error, sensitif outlier)")
    print(f"  • R²: {row['R2']:.3f} ({row['R2']*100:.1f}% variasi dijelaskan)")

    if not pd.isna(row['MAPE']):
        print(f"  • MAPE: {row['MAPE']:.1f}% (rata-rata error persentase)")

    # Business recommendation
    if row['MAE'] < 0.4:  # Less than $40,000 error
        accuracy_level = "Sangat Akurat"
    elif row['MAE'] < 0.6:  # Less than $60,000 error
        accuracy_level = "Akurat"
    elif row['MAE'] < 0.8:  # Less than $80,000 error
        accuracy_level = "Cukup Akurat"
    else:
        accuracy_level = "Kurang Akurat"

    print(f"  {accuracy_level} untuk prediksi harga rumah")

In [ ]:
# =============================================
# BAGIAN 4: CROSS-VALIDATION DAN ROBUST EVALUATION
# =============================================

print("\n BAGIAN 4: CROSS-VALIDATION DAN ROBUST EVALUATION")
print("=" * 60)

In [ ]:
# Langkah 9: K-Fold Cross Validation

print("K-FOLD CROSS VALIDATION UNTUK REGRESI")

# Setup cross-validation
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Metrics untuk cross-validation
cv_results = {
    'Model': [],
    'CV_MAE_mean': [],
    'CV_MAE_std': [],
    'CV_RMSE_mean': [],
    'CV_RMSE_std': [],
    'CV_R2_mean': [],
    'CV_R2_std': []
}

print(f" Menjalankan {k_folds}-Fold Cross Validation...")
print("-" * 50)

for name, model in models.items():
    print(f"Cross-validating {name}...")

    # Lists untuk menyimpan scores setiap fold
    mae_scores, rmse_scores, r2_scores = [], [], []

    # Cross-validation
    for train_idx, val_idx in kf.split(X):
        # Split data
        X_train_cv, X_val_cv = X.iloc[train_idx], X.iloc[val_idx]
        y_train_cv, y_val_cv = y.iloc[train_idx], y.iloc[val_idx]

        # Scale data
        X_train_cv_scaled = scaler.fit_transform(X_train_cv)
        X_val_cv_scaled = scaler.transform(X_val_cv)

        # Train model
        model_cv = model.__class__(**model.get_params())
        model_cv.fit(X_train_cv_scaled, y_train_cv)

        # Predict
        y_pred_cv = model_cv.predict(X_val_cv_scaled)

        # Calculate metrics
        mae_scores.append(mean_absolute_error(y_val_cv, y_pred_cv))
        rmse_scores.append(np.sqrt(mean_squared_error(y_val_cv, y_pred_cv)))
        r2_scores.append(r2_score(y_val_cv, y_pred_cv))

    # Store results
    cv_results['Model'].append(name)
    cv_results['CV_MAE_mean'].append(np.mean(mae_scores))
    cv_results['CV_MAE_std'].append(np.std(mae_scores))
    cv_results['CV_RMSE_mean'].append(np.mean(rmse_scores))
    cv_results['CV_RMSE_std'].append(np.std(rmse_scores))
    cv_results['CV_R2_mean'].append(np.mean(r2_scores))
    cv_results['CV_R2_std'].append(np.std(r2_scores))

    print(f"  {name} - CV R²: {np.mean(r2_scores):.4f} (±{np.std(r2_scores):.4f})")

# Convert ke DataFrame
cv_results_df = pd.DataFrame(cv_results)
cv_results_df = cv_results_df.sort_values('CV_R2_mean', ascending=False)

print("\n CROSS-VALIDATION RESULTS:")
print("=" * 70)
print(cv_results_df.to_string(index=False))

# Visualisasi cross-validation results
plt.figure(figsize=(15, 10))

metrics_cv = ['CV_MAE_mean', 'CV_RMSE_mean', 'CV_R2_mean']
errors_cv = ['CV_MAE_std', 'CV_RMSE_std', 'CV_R2_std']
titles = ['MAE (Cross-Validation)', 'RMSE (Cross-Validation)', 'R² (Cross-Validation)']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, (metric, error, title, color) in enumerate(zip(metrics_cv, errors_cv, titles, colors)):
    plt.subplot(2, 2, idx+1)

    # Sort data
    if 'R2' in metric:
        sorted_idx = cv_results_df[metric].argsort()[::-1]  # Descending for R²
    else:
        sorted_idx = cv_results_df[metric].argsort()  # Ascending for error metrics

    x_pos = range(len(cv_results_df))

    # Plot dengan error bars
    plt.barh(x_pos, cv_results_df[metric].iloc[sorted_idx],
             xerr=cv_results_df[error].iloc[sorted_idx],
             color=color, alpha=0.7, ecolor='black', capsize=5)

    plt.yticks(x_pos, cv_results_df['Model'].iloc[sorted_idx])
    plt.xlabel(f'{title.split("(")[0].strip()} Score')
    plt.title(f'{title}\n(Mean ± Std Dev)')
    plt.grid(True, alpha=0.3, axis='x')

    # Tambahkan nilai
    for i, (v, e) in enumerate(zip(cv_results_df[metric].iloc[sorted_idx],
                                  cv_results_df[error].iloc[sorted_idx])):
        if 'R2' in metric:
            plt.text(v + 0.01, i, f'{v:.3f}±{e:.3f}', va='center', fontsize=8)
        else:
            plt.text(v + e + 0.01, i, f'{v:.3f}±{e:.3f}', va='center', fontsize=8)

# Plot 4: Comparison single test vs CV
plt.subplot(2, 2, 4)
# Gabungkan hasil
comparison_df = pd.merge(results_df, cv_results_df, on='Model')

# Plot R² comparison
x = range(len(comparison_df))
width = 0.35

plt.bar([i - width/2 for i in x], comparison_df['R2'], width,
        label='Single Test Set', alpha=0.8, color='#95E1D3')
plt.bar([i + width/2 for i in x], comparison_df['CV_R2_mean'], width,
        label='CV Mean', alpha=0.8, color='#F38181')

plt.xlabel('Model')
plt.ylabel('R² Score')
plt.title('Comparison: Single Test vs Cross-Validation')
plt.xticks(x, comparison_df['Model'], rotation=45, ha='right')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Statistical comparison of models
print("\n STATISTICAL COMPARISON OF MODELS")
print("=" * 60)

# Paired t-test untuk perbandingan model
from scipy import stats

print("PAIRED T-TEST UNTUK PERBANDINGAN MODEL:")
print("(Membandingkan R² scores antar model berdasarkan CV)")

# Ambil 3 model terbaik
top_models = cv_results_df.head(3)['Model'].tolist()

for i in range(len(top_models)):
    for j in range(i+1, len(top_models)):
        model1 = top_models[i]
        model2 = top_models[j]

        # Ambil R² scores untuk kedua model
        model1_idx = cv_results_df[cv_results_df['Model'] == model1].index[0]
        model2_idx = cv_results_df[cv_results_df['Model'] == model2].index[0]

        # Untuk t-test, kita perlu scores individual, bukan mean
        # Simulasi: generate data dari distribusi normal berdasarkan mean dan std
        np.random.seed(42)
        model1_scores = np.random.normal(
            cv_results_df.loc[model1_idx, 'CV_R2_mean'],
            cv_results_df.loc[model1_idx, 'CV_R2_std'],
            1000
        )
        model2_scores = np.random.normal(
            cv_results_df.loc[model2_idx, 'CV_R2_mean'],
            cv_results_df.loc[model2_idx, 'CV_R2_std'],
            1000
        )

        # Paired t-test
        t_stat, p_value = stats.ttest_rel(model1_scores, model2_scores)

        print(f"\n{model1} vs {model2}:")
        print(f"  t-statistic: {t_stat:.4f}")
        print(f"  p-value: {p_value:.4f}")

        if p_value < 0.05:
            if cv_results_df.loc[model1_idx, 'CV_R2_mean'] > cv_results_df.loc[model2_idx, 'CV_R2_mean']:
                print(f"  {model1} secara signifikan lebih baik dari {model2} (p < 0.05)")
            else:
                print(f"  {model2} secara signifikan lebih baik dari {model1} (p < 0.05)")
        else:
            print(f"  Tidak ada perbedaan signifikan antara {model1} dan {model2}")

In [ ]:
# =============================================
# BAGIAN 5: FINAL MODEL SELECTION & DEPLOYMENT
# =============================================

print("\n BAGIAN 5: FINAL MODEL SELECTION & DEPLOYMENT PREPARATION")
print("=" * 60)

In [ ]:
# Langkah 10: Model Selection dan Final Evaluation

print(" FINAL MODEL SELECTION")

# Gabungkan semua hasil
final_results = pd.merge(results_df, cv_results_df, on='Model')

# Hitung composite score
final_results['Composite_Score'] = (
    final_results['R2'] * 0.25 +
    final_results['CV_R2_mean'] * 0.35 +
    (1 - final_results['CV_R2_std']) * 0.15 +
    (1 - final_results['RMSE']) * 0.15 +
    (1 - final_results['CV_RMSE_mean']) * 0.10
)

# Sort berdasarkan composite score
final_results = final_results.sort_values('Composite_Score', ascending=False)

print("\n FINAL COMPARISON TABLE:")
print("=" * 90)
display_cols = ['Model', 'R2', 'CV_R2_mean', 'CV_R2_std',
                'RMSE', 'CV_RMSE_mean', 'Composite_Score']
display_final = final_results[display_cols].copy()

for col in display_cols[1:]:
    display_final[col] = display_final[col].apply(lambda x: f"{x:.4f}")

print(display_final.to_string(index=False))

# Best model berdasarkan berbagai kriteria
best_overall = final_results.iloc[0]['Model']
best_single_test = final_results.sort_values('R2', ascending=False).iloc[0]['Model']
best_cv = final_results.sort_values('CV_R2_mean', ascending=False).iloc[0]['Model']
best_stable = final_results.sort_values('CV_R2_std', ascending=True).iloc[0]['Model']

print(f"\n  BEST MODEL BY CRITERIA:")
print(f"• Best Overall (Composite Score): {best_overall}")
print(f"• Best Single Test R²: {best_single_test}")
print(f"• Best CV R²: {best_cv}")
print(f"• Most Stable (Lowest CV std): {best_stable}")

# Final model selection
if best_overall == best_cv:
    selected_model_name = best_overall
    selection_reason = "Konsisten performa tinggi di semua evaluasi"
else:
    selected_model_name = best_cv
    selection_reason = "Lebih robust berdasarkan cross-validation"

print(f"\n FINAL SELECTION: {selected_model_name}")
print(f"   Alasan: {selection_reason}")

# Train final model dengan semua training data
print(f"\n TRAINING FINAL MODEL: {selected_model_name}")
selected_model = trained_models[selected_model_name].__class__(
    **trained_models[selected_model_name].get_params()
)

# Train dengan semua data yang tersedia
X_final = pd.concat([X_train, X_test])
y_final = pd.concat([y_train, y_test])

# Scale data
X_final_scaled = scaler.fit_transform(X_final)

# Train model
selected_model.fit(X_final_scaled, y_final)

print("Final model berhasil ditraining dengan semua data!")

In [ ]:
# Langkah 11: Model Interpretation dan Feature Importance

print("\n MODEL INTERPRETATION & FEATURE IMPORTANCE")

# Feature importance untuk tree-based models
if hasattr(selected_model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': selected_model.feature_importances_
    }).sort_values('Importance', ascending=False)

    print("\n FEATURE IMPORTANCE:")
    print(feature_importance.to_string(index=False))

    # Visualisasi feature importance
    plt.figure(figsize=(10, 6))
    plt.barh(range(len(feature_importance)),
             feature_importance['Importance'].values[::-1])
    plt.yticks(range(len(feature_importance)),
               feature_importance['Feature'].values[::-1])
    plt.xlabel('Importance Score')
    plt.title(f'Feature Importance - {selected_model_name}')
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

# Coefficients untuk linear models
elif hasattr(selected_model, 'coef_'):
    coefficients = pd.DataFrame({
        'Feature': X.columns,
        'Coefficient': selected_model.coef_
    }).sort_values('Coefficient', ascending=False)

    print("\n MODEL COEFFICIENTS:")
    print(coefficients.to_string(index=False))

    # Visualisasi coefficients
    plt.figure(figsize=(10, 6))
    colors = ['red' if c < 0 else 'green' for c in coefficients['Coefficient']]
    plt.barh(range(len(coefficients)), coefficients['Coefficient'].values[::-1],
             color=colors[::-1])
    plt.yticks(range(len(coefficients)), coefficients['Feature'].values[::-1])
    plt.xlabel('Coefficient Value')
    plt.title(f'Model Coefficients - {selected_model_name}')
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    plt.grid(True, alpha=0.3, axis='x')
    plt.tight_layout()
    plt.show()

# Interpretasi business
print("\n BUSINESS INTERPRETATION:")

# Contoh interpretasi untuk California Housing
if 'MedInc' in X.columns:
    print("Berdasarkan model yang dipilih:")
    print("1. **Pendapatan Median (MedInc)**: Faktor paling penting")
    print("   • Setiap kenaikan 1 unit MedInc meningkatkan harga rumah sekitar X")
    print("   • Investasi di area dengan pendapatan tinggi memberikan ROI lebih baik")

if 'AveRooms' in X.columns:
    print("\n2. **Rata-rata Kamar (AveRooms)**:")
    print("   • Rumah dengan lebih banyak kamar cenderung lebih mahal")
    print("   • Tapi hati-hati dengan rumah yang terlalu besar (mungkin kurang efisien)")

if 'Latitude' in X.columns and 'Longitude' in X.columns:
    print("\n3. **Lokasi (Latitude, Longitude)**:")
    print("   • Lokasi sangat penting dalam menentukan harga rumah")
    print("   • Dekat dengan pusat kota atau pantai biasanya lebih mahal")

print("\n RECOMMENDATIONS FOR HOUSING MARKET:")
print("• Fokus pada area dengan pendapatan median tinggi")
print("• Pertimbangkan ukuran rumah yang optimal (bukan maksimal)")
print("• Lokasi adalah faktor kritis - riset area secara mendalam")
print("• Gunakan model ini untuk price estimation dan investment analysis")

In [ ]:
# Langkah 12: Export Model dan Final Report

print("\n EXPORTING MODEL & FINAL REPORT")
print("=" * 60)

import pickle
import json
from datetime import datetime

# Create results directory
import os
os.makedirs('regression_results', exist_ok=True)

# 1. Save the trained model
model_filename = f'regression_results/best_model_{selected_model_name.replace(" ", "_")}.pkl'
with open(model_filename, 'wb') as f:
    pickle.dump(selected_model, f)
print(f" Model saved to: {model_filename}")

# 2. Save the scaler
scaler_filename = 'regression_results/scaler.pkl'
with open(scaler_filename, 'wb') as f:
    pickle.dump(scaler, f)
print(f" Scaler saved to: {scaler_filename}")

# 3. Save feature names
feature_names = X.columns.tolist()
with open('regression_results/feature_names.json', 'w') as f:
    json.dump(feature_names, f)
print("Feature names saved")

# 4. Create comprehensive report
report = {
    'project': 'California Housing Price Prediction',
    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'selected_model': selected_model_name,
    'selection_reason': selection_reason,
    'final_performance': {
        'R2': float(final_results[final_results['Model'] == selected_model_name]['R2'].iloc[0]),
        'CV_R2_mean': float(final_results[final_results['Model'] == selected_model_name]['CV_R2_mean'].iloc[0]),
        'CV_R2_std': float(final_results[final_results['Model'] == selected_model_name]['CV_R2_std'].iloc[0]),
        'RMSE': float(final_results[final_results['Model'] == selected_model_name]['RMSE'].iloc[0]),
        'MAE': float(final_results[final_results['Model'] == selected_model_name]['MAE'].iloc[0]),
        'MAPE': float(final_results[final_results['Model'] == selected_model_name]['MAPE'].iloc[0]) if not pd.isna(final_results[final_results['Model'] == selected_model_name]['MAPE'].iloc[0]) else None
    },
    'dataset_info': {
        'n_samples': len(X_final),
        'n_features': X_final.shape[1],
        'target_mean': float(y_final.mean()),
        'target_std': float(y_final.std())
    },
    'business_interpretation': {
        'average_error_dollars': float(final_results[final_results['Model'] == selected_model_name]['MAE'].iloc[0] * 100000),
        'error_range_dollars': f"${final_results[final_results['Model'] == selected_model_name]['RMSE'].iloc[0] * 100000:.0f}",
        'accuracy_level': "Sangat Akurat" if final_results[final_results['Model'] == selected_model_name]['MAE'].iloc[0] < 0.4 else
                         "Akurat" if final_results[final_results['Model'] == selected_model_name]['MAE'].iloc[0] < 0.6 else
                         "Cukup Akurat" if final_results[final_results['Model'] == selected_model_name]['MAE'].iloc[0] < 0.8 else
                         "Kurang Akurat"
    }
}

# Save report
with open('regression_results/final_report.json', 'w') as f:
    json.dump(report, f, indent=4)
print("Final report saved")

# 5. Create summary CSV
summary_df = final_results.copy()
summary_df.to_csv('regression_results/model_comparison_summary.csv', index=False)
print("Model comparison summary saved")

# 6. Download all files
from google.colab import files
import zipfile

# Create zip file
with zipfile.ZipFile('regression_results.zip', 'w') as zipf:
    for root, dirs, file_name in os.walk('regression_results'): # Changed 'files' to 'file_name'
        for f_name in file_name:
            zipf.write(os.path.join(root, f_name),
                      os.path.relpath(os.path.join(root, f_name),
                                     os.path.join('regression_results', '..')))

print("\n ALL FILES PACKAGED INTO: regression_results.zip")
print("\n FILES CREATED:")
print("1. best_model_*.pkl - Trained model untuk deployment")
print("2. scaler.pkl - Scaler untuk preprocessing")
print("3. feature_names.json - Nama features untuk inference")
print("4. final_report.json - Laporan komprehensif hasil analisis")
print("5. model_comparison_summary.csv - Perbandingan semua model")

# Download the zip file
files.download('regression_results.zip')

print("\n ANALISIS REGRESI SELESAI!")
print("Model telah dievaluasi, dipilih, dan siap untuk deployment")
print("Laporan lengkap telah dibuat dan didownload")

In [ ]:
# Langkah 13: Contoh Penggunaan Model untuk Prediksi Baru

print("\n CONTOH DEPLOYMENT & PREDIKSI BARU")
print("=" * 60)

# Contoh data baru untuk prediksi
print("CONTOH DATA BARU UNTUK PREDIKSI:")
new_data_example = pd.DataFrame({
    'MedInc': [8.0, 3.0, 5.0],
    'HouseAge': [30, 15, 25],
    'AveRooms': [6.0, 4.0, 5.0],
    'AveBedrms': [1.2, 1.0, 1.1],
    'Population': [2000, 1000, 1500],
    'AveOccup': [3.0, 2.5, 2.8],
    'Latitude': [37.8, 34.0, 36.5],
    'Longitude': [-122.4, -118.2, -119.7]
})

print(new_data_example)

# Fungsi untuk prediksi
def predict_new_data(model, scaler, new_data):
    """Predict house prices for new data"""
    # Pastikan urutan features sama dengan training
    new_data = new_data[X.columns]

    # Scale data
    new_data_scaled = scaler.transform(new_data)

    # Predict
    predictions = model.predict(new_data_scaled)

    return predictions

# Contoh prediksi
try:
    predictions = predict_new_data(selected_model, scaler, new_data_example)

    print("\n HASIL PREDIKSI:")
    for i, (idx, row) in enumerate(new_data_example.iterrows()):
        print(f"\nData {i+1}:")
        print(f"  Features: MedInc={row['MedInc']:.1f}, HouseAge={row['HouseAge']}, Rooms={row['AveRooms']:.1f}")
        print(f"  Predicted Price: ${predictions[i]*100000:,.0f}")
        print(f"  Confidence Interval: ±${final_results[final_results['Model'] == selected_model_name]['RMSE'].iloc[0]*100000:,.0f}")

        # Business interpretation
        if predictions[i] > y_final.mean():
            price_level = "DI ATAS rata-rata"
        else:
            price_level = "DI BAWAH rata-rata"

        print(f"  Interpretasi: Harga prediksi {price_level} pasar")

except Exception as e:
    print(f" Error dalam prediksi: {str(e)}")

print("\n" + "="*60)
print("SELAMAT! Anda telah menyelesaikan evaluasi model regresi lengkap")
print("Materi pertemuan 13 selesai: MAE, MSE, RMSE, R², MAPE, CV, Residual Analysis")
print("Model siap digunakan untuk prediksi harga rumah di California!")